# Librerias

In [11]:
import time
import random
from datetime import datetime, timedelta
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException

# Configuración

In [ ]:
# URL de acceso a FusionSolar Monitorización
url_login = "https://eu5.fusionsolar.huawei.com/unisso/login.action"

# Credenciales (sustituye por tus valores reales)
email = "XXXXXXX"
password = "XXXXXXX"

# Definir el nombre de la planta
nombre_planta = "XXXXXXXXX"

# Definir rango de fechas (YYYY-MM-DD)
fecha_inicio = datetime.strptime("2025-01-01", "%Y-%m-%d")  
fecha_fin    = datetime.strptime("2025-01-31", "%Y-%m-%d")  # inclusive

## Inicio del navegador

In [ ]:
# Configuración del driver (Chrome)
options = webdriver.ChromeOptions()
# Si quieres ejecutar en modo headless, descomenta la siguiente línea:
# options.add_argument("--headless")
driver = webdriver.Chrome(options=options)

# Timeout por defecto para esperas explícitas y evitar baneos
timeout = 10



# Página de acceso

In [4]:
# Celda – Navegar a la página de login y rellenar usuario/contraseña (mejorando las esperas)

# 1. Abrir la URL de login
driver.get(url_login)

# 2. Esperar a que el campo de usuario esté clickable y visible
WebDriverWait(driver, timeout).until(
    EC.element_to_be_clickable((By.ID, "username"))
)
input_email = driver.find_element(By.ID, "username")
# Hacer click primero para asegurarnos de que el campo está activo
input_email.click()
input_email.send_keys(email)

# 3. Esperar a que el campo de contraseña esté clickable y visible
WebDriverWait(driver, timeout).until(
    EC.element_to_be_clickable((By.ID, "value"))
)
input_pwd = driver.find_element(By.ID, "value")
input_pwd.click()
input_pwd.send_keys(password)

# 4. Esperar a que el botón de login (reemplaza submitDataverify si es otro ID o selector) esté clickable
WebDriverWait(driver, timeout).until(
    EC.element_to_be_clickable((By.ID, "submitDataverify"))
)
driver.find_element(By.ID, "submitDataverify").click()


# Trabajando en el dashboard

## Filtrado por nombre de planta

In [5]:
# Celda – Buscar planta en el dashboard y abrir el enlace resultante


# 1. Intentar localizar el campo de búsqueda directamente
try:
    WebDriverWait(driver, timeout).until(
        EC.visibility_of_element_located((By.ID, "searchName"))
    )
    input_search = driver.find_element(By.ID, "searchName")
except TimeoutException:
    # 2. Si no aparece, es probable que esté dentro de un <iframe>
    #    Buscamos todos los iframes, cambiamos a cada uno y probamos de nuevo
    iframes = driver.find_elements(By.TAG_NAME, "iframe")
    for frame in iframes:
        driver.switch_to.frame(frame)
        try:
            WebDriverWait(driver, timeout).until(
                EC.visibility_of_element_located((By.ID, "searchName"))
            )
            input_search = driver.find_element(By.ID, "searchName")
            break  # Salimos del bucle si lo encontramos
        except TimeoutException:
            driver.switch_to.default_content()
    else:
        raise Exception("No se encontró el campo de búsqueda 'searchName' ni en iframes ni en el DOM principal.")

# 3. Una vez tenemos input_search, hacemos clic y escribimos
input_search.click()
input_search.clear()
input_search.send_keys(nombre_planta)
input_search.send_keys(Keys.ENTER)

# 4. Volver al contenido principal antes de buscar el enlace (si estábamos en un iframe)
driver.switch_to.default_content()

# 5. Esperar a que aparezca el enlace con el texto de la planta y hacer clic
WebDriverWait(driver, timeout).until(
    EC.element_to_be_clickable((By.LINK_TEXT, nombre_planta))
)
driver.find_element(By.LINK_TEXT, nombre_planta).click()


## Gestión de informes de la planta

In [6]:
# Celda – Hacer clic en la pestaña "Gestión de informes"

# (Revisa que Selenium y las variables driver/timeout ya estén definidos en celdas anteriores)

WebDriverWait(driver, timeout).until(
    EC.element_to_be_clickable((By.LINK_TEXT, "Gestión de informes"))
)
driver.find_element(By.LINK_TEXT, "Gestión de informes").click()

### Selección del dia

In [ ]:
# 2. Seleccionar “Diario” en el dropdown de granularidad (se hace una sola vez)
WebDriverWait(driver, timeout).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "div.ant-select-selector"))
)
driver.find_element(By.CSS_SELECTOR, "div.ant-select-selector").click()
WebDriverWait(driver, timeout).until(
    EC.element_to_be_clickable((By.XPATH, "//span[@title='Diario']"))
)
driver.find_element(By.XPATH, "//span[@title='Diario']").click()

# 3. Recorrer cada día del rango
datos_completos = []
fecha_actual = fecha_inicio
primera_iteracion = True

while fecha_actual <= fecha_fin:
    fecha_str = fecha_actual.strftime("%Y-%m-%d")
    
    # 3.1. Poner la fecha en el campo “Periodo estadístico”
    WebDriverWait(driver, timeout).until(
        EC.element_to_be_clickable((By.ID, "statisticTime"))
    )
    input_fecha = driver.find_element(By.ID, "statisticTime")
    input_fecha.click()
    # Seleccionar todo el texto existente y borrarlo
    input_fecha.send_keys(Keys.CONTROL, "a")
    input_fecha.send_keys(Keys.DELETE)
    # Escribir la nueva fecha y pulsar Enter
    input_fecha.send_keys(fecha_str)
    input_fecha.send_keys(Keys.ENTER)
    
    # 3.2. Clic en “Buscar”
    WebDriverWait(driver, timeout).until(
        EC.element_to_be_clickable((By.XPATH, "//button[normalize-space()='Buscar']"))
    )
    driver.find_element(By.XPATH, "//button[normalize-space()='Buscar']").click()
    
    # 3.3. Esperar a que cargue la tabla diaria (al menos un <tr> está presente)
    WebDriverWait(driver, timeout).until(
        EC.presence_of_all_elements_located((By.XPATH, "//tbody//tr"))
    )
    
    # 3.4. Ajustar paginación a 50 filas solo en la primera iteración
    if primera_iteracion:
        # Detectar el control visible de paginación (“10 / página” o “50 / página”)
        pag_span = WebDriverWait(driver, timeout).until(
            EC.visibility_of_element_located((By.XPATH, "//span[contains(@title,'/ página')]"))
        )
        titulo_actual = pag_span.get_attribute("title").strip()
        if titulo_actual == "10 / página":
            pag_span.click()
            # Esperar a que aparezca la opción “50 / página” y hacer clic
            opcion_50 = WebDriverWait(driver, timeout).until(
                EC.element_to_be_clickable((
                    By.XPATH,
                    "//div[contains(@class,'ant-select-item-option-content') and normalize-space(text())='50 / página']"
                ))
            )
            opcion_50.click()
            # Esperar recarga de la tabla tras cambiar a 50 filas
            WebDriverWait(driver, timeout).until(
                EC.presence_of_all_elements_located((By.XPATH, "//tbody//tr"))
            )
        primera_iteracion = False
    
    # 3.5. Leer encabezados de la tabla
    ths = driver.find_elements(By.XPATH, "//thead//th")
    columnas = [th.text.strip() for th in ths]
    
    # 3.6. Localizar de nuevo las filas justo antes de leerlas
    filas_actuales = driver.find_elements(By.XPATH, "//tbody//tr")

    for fila in filas_actuales:
        # Leemos de golpe los textos de las celdas sin guardar referencias a WebElements a largo plazo
        textos = [td.text.strip() for td in fila.find_elements(By.TAG_NAME, "td")]
        fila_dict = dict(zip(columnas, textos))
        #fila_dict["FechaConsulta"] = fecha_str
        datos_completos.append(fila_dict)


    # 3.7. Avanzar al siguiente día
    time.sleep(random.uniform(5, 20)) # Se añaden pausas aleatorias para evitar baneos
    fecha_actual += timedelta(days=1)

# 4. Crear DataFrame con todo el rango
df_rango_completo = pd.DataFrame(datos_completos)
# Eliminar filas con "Período estadístico" en blanco
df_rango_completo = df_rango_completo[df_rango_completo["Período estadístico"].astype(bool)]
# Eliminar columnas sin nombre (Unnamed o cadena vacía)
df_rango_completo = df_rango_completo.loc[:, df_rango_completo.columns.str.strip() != ""]
df_rango_completo


,Período estadístico,Rendimiento FV (kWh),Rendimiento del inversor (kWh),Energía exportada (kWh),Ingresos (€),FechaConsulta
1,2025-01-01 00:00:00,"0,000","0,000","0,000","0,00",2025-01-01
2,2025-01-01 01:00:00,--,--,"0,000","0,00",2025-01-01
3,2025-01-01 02:00:00,--,--,"0,000","0,00",2025-01-01
4,2025-01-01 03:00:00,--,--,"0,000","0,00",2025-01-01
5,2025-01-01 04:00:00,--,--,"0,000","0,00",2025-01-01
...,...,...,...,...,...,...
950,2025-01-31 19:00:00,"0,000","0,000","0,000","0,00",2025-01-31
951,2025-01-31 20:00:00,--,--,"0,000","0,00",2025-01-31
952,2025-01-31 21:00:00,--,--,"0,000","0,00",2025-01-31
953,2025-01-31 22:00:00,--,--,"0,000","0,00",2025-01-31


# Guardado en CSV

In [17]:
# Formatear las fechas solo con mes y año para el nombre de archivo
inicio_str = fecha_inicio.strftime("%m-%Y")
fin_str    = fecha_fin.strftime("%m-%Y")

# Construir el nombre sin caracteres inválidos
nombre_archivo = f"{nombre_planta}_{inicio_str}_{fin_str}.csv"

# Guardar el DataFrame
df_rango_completo.to_csv(nombre_archivo, index=False)

In [13]:

df_rango_completo.head(25)

,Período estadístico,Rendimiento FV (kWh),Rendimiento del inversor (kWh),Energía exportada (kWh),Ingresos (€),FechaConsulta
1,2025-01-01 00:00:00,"0,000","0,000","0,000","0,00",2025-01-01
2,2025-01-01 01:00:00,--,--,"0,000","0,00",2025-01-01
3,2025-01-01 02:00:00,--,--,"0,000","0,00",2025-01-01
4,2025-01-01 03:00:00,--,--,"0,000","0,00",2025-01-01
5,2025-01-01 04:00:00,--,--,"0,000","0,00",2025-01-01
6,2025-01-01 05:00:00,--,--,"0,000","0,00",2025-01-01
7,2025-01-01 06:00:00,--,--,"0,000","0,00",2025-01-01
8,2025-01-01 07:00:00,--,--,"0,000","0,00",2025-01-01
9,2025-01-01 08:00:00,"0,130","0,130","0,000","0,00",2025-01-01
10,2025-01-01 09:00:00,"2,990","2,990","0,000","0,00",2025-01-01
